# Chaper 1 Schema and Types

In order to provide access to the underlying database, SQLAlchemy needs a representation of the tables that should be present in the database. We can do this in one of three ways:
- Using user-defined `table` objects
- Using declarative classes that represent your tables
- Inferring them from the database

## 1.0 Types

There are four categories of types we can use inside of SQLAlchemy:
- Generic
- SQL standard
- Vendor specific
- User defined

SQLAlchemy defines a large number of generic types that are abstracted away from
the actual SQL types supported by each backend database. These types are all avail‐
able in the `sqlalchemy.types` module, and for convenience they are also available in
the sqlalchemy module. So let’s think about how these generic types are useful.

See https://docs.sqlalchemy.org/en/13/core/type_basics.html for overview.

You can use generic SQL types, somethin i believe i would recommend as the SQL Database can be already created beforehand. Probably also good if one wish to work with PostgreSQL. These are available thought `sqlalchemy.types` e.g. `sqlalchemy.BIGINT` or `sqlalchemy.VARCHAR`

### 1.0.1 Dialects
The **dialect** is the system SQLAlchemy uses to communicate with various types of DBAPI implementations and databases. The sections that follow contain reference documentation and notes specific to the usage of each backend, as well as notes for the various DBAPIs.

see: https://docs.sqlalchemy.org/en/14/dialects/

These are all available in the `sqlalchemy.dialect` module e.g.:

In [1]:
from sqlalchemy.dialects.postgresql import JSON

Now we can define JSON fields that we can later use with the many PostgreSQL-
specific JSON functions, such as `array_to_json` , within our application.

## 1.1 Metadata
Metadata is used to tie together the database structure so it can be quickly accessed
inside SQLAlchemy. It’s often useful to think of metadata as a kind of catalog of `Table`
objects with optional information about the engine and the connection. Those tables
can be accessed via a dictionary, `MetaData.tables` .
**Metadata needs to be imported and initialized before objects can be tied to it.** Let’s initialize an instance of the `MetaData` objects that we can use throughout the rest of the examples in this chapter to hold our information catalog:

In [2]:
from sqlalchemy import MetaData
metadata = MetaData()

## 1.2 Tables
Table objects are initialized in SQLAlchemy Core in a supplied `MetaData` object by
calling the `Table` constructor with the table name and metadata.The columns are constructed by calling `Column` with a name, type, and then arguments that represent any additional SQL constructs and constraints. We create a table that could be used to store the cookie inventory for our online cookie delivery service.

In [3]:
from sqlalchemy import Table, Column, Integer, Numeric, String, ForeignKey, Boolean
from sqlalchemy.exc import InvalidRequestError

try: 
    cookies = Table("cookies", metadata,
        Column("cookie_id", Integer(), primary_key=True),
        Column('cookie_name', String(50), index=True), # Indexing this column to speed up queries
        Column('cookie_recipe_url', String(255)),
        Column('cookie_sku', String(55)),
        Column('quantity', Integer()),
        Column('unit_cost', Numeric(12, 2))
    )
except InvalidRequestError:
    pass

## 1.3 Columns
Colums define the fields that exists in our tables, and they provide the primary means by which we define other constraints through their keyword arguments. We can mark columns as required and/or force them to be unique. We can also set default initial values and change values when the record is updated.

In [4]:
from datetime import datetime
from sqlalchemy import DateTime

try:
    users = Table('users', metadata,
        Column('user_id', Integer(), primary_key=True),
        Column('username', String(15), nullable=False, unique=True),
        Column('email_address', String(255), nullable=False),
        Column('phone', String(20), nullable=False),
        Column('password', String(25), nullable=False),
        Column('created_on', DateTime(), default=datetime.now),
        Column('updated_on', DateTime(), default=datetime.now, onupdate=datetime.now)
    )
    # Using onupdate=datetime.now will uptade the column to the date it was updated.
    # using nullable=False is the same as NOT NULL in sql and uniqe=True is self explanatory.
except InvalidRequestError:
    pass

## 1.4 Keys and Constraints
Keys and constraints are used as a way to ensure that our data meets certain require‐
ments prior to being stored in the database. The objects that represent keys and con‐
straints can be found inside the base SQLAlchemy module, and three of the more
common ones can be imported as shown here:

In [5]:
from sqlalchemy import PrimaryKeyConstraint, UniqueConstraint, CheckConstraint

The `PrimaryKeyConstraint` means making a column a Primary key, basicly a tuple, you can set it when creating the table as shown in **1.3** but also in hindsight of creating the table. For composite primary keys simply use commas:

In [6]:
# PrimaryKeyConstraint("User_id", name = "user_pk")

To define a `UniqueConstraint` follows the same formula as above, and can be done externally by using:

In [7]:
# UniqueConstraint("username", name = "uix_username")

Not shown before is the `CheckConstraint` type. This type of constraint is used
to ensure that the data supplied for a column matches a set of user-defined criteria. In
the following example, we are ensuring that `unit_cost` is never allowed to be less
than 0.00 because every cookie costs something to make:

In [8]:
CheckConstraint("unit_cost >= 0.00", name="unit_cost_positive")

CheckConstraint(<sqlalchemy.sql.elements.TextClause object at 0x7feb78fe4130>, name='unit_cost_positive')

## 1.5 Indexes
Indexes are used to accelerate lookups for field values. When indexes are created as shown in the example in **1.2**, you will have an index called ix_cookies_cookie_name. We can assign it afterwards aswell as follows:

In [9]:
from sqlalchemy import Index
#Index("ix_cookies_cookie_name", "cookie_name")

We can also create functional indexes that vary a bit by the backend database being
used. This lets you create an index for situations where you often need to query based
on some unusual context. For example, what if we want to select by cookie SKU and
name as a joined item, such as *SKU0001 Chocolate Chip*? We could define an index
like this to optimize that lookup:

In [10]:
#Index("ix_test", mytable.c.cookie_sku, mytable.c.cookie_name)

## Relationships and ForeignKeyConstraints
Now that we have a table with columns with all the right constraints and indexes, let’s
look at how we create relationships between tables. We need a way to track orders,
including line items that represent each cookie and quantity ordered.

Association tables are used to enable many-to-many relationships between two other tables. A single `ForeignKey` on a table is typically a sign of a one-to-many relationship; however, if there are multiple `ForeignKey` relationships on a table, there is a strong possibility that it is an association table.

You can look into assosiation tables later on. For now lets build our two assosiation tables to connect the previously made tables:

In [11]:
from sqlalchemy import ForeignKey

try: 
    orders = Table('orders', metadata,
        Column('order_id', Integer(), primary_key=True),
        Column('user_id', ForeignKey('users.user_id')), # Note the use of a string to define key
        Column('shipped', Boolean(), default=False)
    )
    line_items = Table('line_items', metadata,
        Column('line_items_id', Integer(), primary_key=True),
        Column('order_id', ForeignKey('orders.order_id')),
        Column('cookie_id', ForeignKey('cookies.cookie_id')),
        Column('quantity', Integer()),
        Column('extended_cost', Numeric(12, 2))
    )
    
except InvalidRequestError:
    pass

If we use hard references, such as cookies.c.cookie_id , in our ForeignKey definitions it will perform that resolution during module initialization and could fail depending on the
order in which the tables are loaded.

To define a `ForeignKeyConstraint` externally we can use:


In [14]:
# ForeignKeyConstraint(['order_id'], ['orders.order_id'])

## Persisting the Tables
All of our tables and additional schema definitions are associated with an instance of
`metadata` . Persisting the schema to the database is simply a matter of calling the
`create_all()` method on our `metadata` instance with the engine where it should create those tables, here i will create them in memory, but moving forward i will most likely use postgreSQL as the engine and remote controll it.

In [19]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:')
metadata.create_all(engine)

**The full pythonscript for this database can be found under `chapter_1_database.py`**